In [1]:
import pandas as pd
import numpy as np

## 1. 부동산거래량 대비 유동인구 데이터

In [2]:
# 분기별 부동산거래량 대비 유동인구
pop2201 = pd.read_csv("부동산거래량_대비_유동인구2201_csv.csv", sep='|') # 22년3월 집계
pop2202 = pd.read_csv("부동산거래량_대비_유동인구2202_csv.csv", sep='|') # 22년 6월 집계
pop2203 = pd.read_csv("부동산거래량_대비_유동인구2203_csv.csv", sep='|') # 22년 9월 집계
pop2204 = pd.read_csv("부동산거래량_대비_유동인구2204_csv.csv", sep='|') # 22년 12월 집계
pop2301 = pd.read_csv("부동산거래량_대비_유동인구2301_csv.csv", sep='|') # 23년 3월 집계

In [3]:
temp1 = pop2201[pop2201['induty_cd'].str.contains('A')]
temp2 = pop2202[pop2202['induty_cd'].str.contains('A')]
temp3 = pop2203[pop2203['induty_cd'].str.contains('A')]
temp4 = pop2204[pop2204['induty_cd'].str.contains('A')]
temp5 = pop2301[pop2301['induty_cd'].str.contains('A')]

In [4]:
temp = pd.concat([temp1, temp2,temp3, temp4, temp5])

In [5]:
temp.drop(['legaldong_cd'], axis=1, inplace=True)

In [6]:
temp.isnull().sum()

data_strd_ym    0
pnu             0
induty_cd       0
fpop_scor       0
dtype: int64

In [7]:
len(temp1)+len(temp2)+len(temp3)+len(temp4)+ len(temp5) # =len(temp)

2562233

## 1-1) 원본 데이터+test 데이터 합친 raw 데이터에 결합

In [8]:
raw_data = pd.read_csv('data_given.csv', encoding='cp949')
match = pd.read_csv('필지번호_행정동.csv', encoding='cp949')
test = pd.read_excel('submission.xlsx')

In [9]:
test_2301 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302

<ipython-input-9-31a165cb8f5b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
<ipython-input-9-31a165cb8f5b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [10]:
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]

In [11]:
raw_data = pd.concat([raw_data,test_2301, test_2302])

In [12]:
temp.rename(columns={'pnu':'LT_UNQ_NO' ,'induty_cd':'INDUSTRY_CD', 'fpop_scor': '부동산거래대비유동인구'}, inplace=True)

In [13]:
temp.reset_index(inplace=True)

In [14]:
conditions = [
    (raw_data['DATA_CRTR_YM'] <= 202203), # 조건1
    ((raw_data['DATA_CRTR_YM'] >= 202204) & (raw_data['DATA_CRTR_YM'] <= 202206)), # 조건2
    ((raw_data['DATA_CRTR_YM'] >= 202207) & (raw_data['DATA_CRTR_YM'] <= 202209)), 
    ((raw_data['DATA_CRTR_YM'] >= 202210) & (raw_data['DATA_CRTR_YM'] <= 202212)), 
    ((raw_data['DATA_CRTR_YM'] >= 202301) & (raw_data['DATA_CRTR_YM'] <= 202303)), 
]

choices = [
    temp[temp['data_strd_ym'] == 202203],  # 조건 1에 대한 선택
    temp[temp['data_strd_ym'] == 202206],  # 조건 2에 대한 선택
    temp[temp['data_strd_ym'] == 202209],
    temp[temp['data_strd_ym'] == 202212],
    temp[temp['data_strd_ym'] == 202303],
]

pop_df = pd.DataFrame()

for i in range(len(conditions)):
    condition = conditions[i]
    selected_rows_A = raw_data[condition]
    selected_rows_B = choices[i]  # 조건에 따른 선택
    pop_df_part = pd.merge(selected_rows_A, selected_rows_B, on=['LT_UNQ_NO', 'INDUSTRY_CD'], how='left')
    pop_df = pd.concat([pop_df, pop_df_part], ignore_index=True)

In [15]:
pop_df # 데이터 손실 없음

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,index,data_strd_ym,부동산거래대비유동인구
0,202201,1111010100100030100,11110101,A03,4.0,1.0,202203.0,19.76
1,202201,1111010100100480000,11110101,A01,2.0,4.0,202203.0,39.13
2,202201,1111010100100590001,11110101,A11,3.0,8.0,202203.0,27.26
3,202201,1111010100100590005,11110101,A03,2.0,10.0,202203.0,21.03
4,202201,1111010100100690000,11110101,A01,2.0,13.0,202203.0,23.93
...,...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,61516.0,202303.0,52.98
1176036,202302,1135010300107450000,11350103,A13,NaN,75809.0,202303.0,36.41
1176037,202302,1150010300103400017,11500103,A06,NaN,119214.0,202303.0,45.98
1176038,202302,1144012700116070000,11440127,A03,NaN,108200.0,202303.0,56.97


In [16]:
pop_df.drop(['index','data_strd_ym'], axis=1, inplace=True)

## 1-2) 필지번호 매칭 안되는 것들 평균 넣어주기

In [19]:
# 'LT_UNQ_NO' 열에서 앞 10자리를 추출
pop_df['LT_UNQ_NO_prefix'] = pop_df['LT_UNQ_NO'].astype(str).str[:10]
temp['LT_UNQ_NO_prefix'] = temp['LT_UNQ_NO'].astype(str).str[:10]

In [20]:
# 값 대체를 위한 groupby mean 계산해서 데이터프레임 생성
sub_df = temp.groupby(['LT_UNQ_NO_prefix', 'data_strd_ym'])['부동산거래대비유동인구'].mean().reset_index()
sub_df

,LT_UNQ_NO_prefix,data_strd_ym,부동산거래대비유동인구
0,1111010100,202203,30.332941
1,1111010100,202206,29.845294
2,1111010100,202209,26.383529
3,1111010100,202212,28.202778
4,1111010100,202303,30.843125
...,...,...,...
52099,5013032026,202203,4.660000
52100,5013032026,202206,4.792727
52101,5013032026,202209,6.461667
52102,5013032026,202212,7.774545


In [21]:
# pop_df에서 필지명이 없어서 '부동산거래대비유동인구'가 결측값인 행들 추출
not_merged = pop_df[pop_df['부동산거래대비유동인구'].isna()]
len(not_merged) # 75796

75796

In [22]:
not_2201 = pd.merge(not_merged[not_merged['DATA_CRTR_YM'] <= 202203], sub_df[sub_df['data_strd_ym']==202203], on='LT_UNQ_NO_prefix', how='left')
not_2202 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202206) & (not_merged['DATA_CRTR_YM'] > 202203)], sub_df[sub_df['data_strd_ym']==202206], on='LT_UNQ_NO_prefix', how='left')
not_2203 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202209) & (not_merged['DATA_CRTR_YM'] > 202206)], sub_df[sub_df['data_strd_ym']==202209], on='LT_UNQ_NO_prefix', how='left')
not_2204 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202212) & (not_merged['DATA_CRTR_YM'] > 202209)], sub_df[sub_df['data_strd_ym']==202212], on='LT_UNQ_NO_prefix', how='left')
not_2301 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202303) & (not_merged['DATA_CRTR_YM'] > 202212)], sub_df[sub_df['data_strd_ym']==202303], on='LT_UNQ_NO_prefix', how='left')

In [23]:
# 분기별 데이터 하나로 합치기
notin_pop = pd.concat([not_2201, not_2202, not_2203, not_2204, not_2301])
notin_pop # 75796

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구_x,LT_UNQ_NO_prefix,data_strd_ym,부동산거래대비유동인구_y
0,202201,1111010200100570000,11110102,A05,4.0,NaN,1111010200,202203.0,35.098571
1,202201,1111010400100530000,11110104,A05,5.0,NaN,1111010400,202203.0,27.218571
2,202201,1111010600100350042,11110106,A01,2.0,NaN,1111010600,202203.0,41.603529
3,202201,1111010600100370000,11110106,A02,4.0,NaN,1111010600,202203.0,41.603529
4,202201,1111010600100490001,11110106,A02,2.0,NaN,1111010600,202203.0,41.603529
...,...,...,...,...,...,...,...,...,...
1321,202302,1117013000101320006,11170130,A03,NaN,NaN,1117013000,202303.0,34.009218
1322,202302,1156012900100610004,11560129,A04,NaN,NaN,1156012900,202303.0,37.916607
1323,202302,1144012400102410098,11440124,A04,NaN,NaN,1144012400,202303.0,44.533655
1324,202302,1130510300105680024,11305103,A02,NaN,NaN,1130510300,202303.0,49.861538


In [24]:
notin_pop.drop(columns='부동산거래대비유동인구_x', axis=1, inplace=True)
notin_pop.rename(columns={'부동산거래대비유동인구_y':'부동산거래대비유동인구'}, inplace=True)

## 1-3) 이 안에서 또 결측값 있어서 시군구 기준으로 평균낸 값 넣기

In [26]:
notin_pop[notin_pop['data_strd_ym'].isna()] # 51개 모두 train 데이터 ... 이것들은 드랍하는게 어떨까.. 아니면 전체 평균 넣어도 되고 

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,LT_UNQ_NO_prefix,data_strd_ym,부동산거래대비유동인구
909,202201,1117010300100040000,11170103,A01,5.0,1117010300,NaN,NaN
910,202201,1117010300100190001,11170103,A01,5.0,1117010300,NaN,NaN
4332,202201,1150011100106760000,11500111,A03,3.0,1150011100,NaN,NaN
4333,202201,1150011100106760000,11500111,A01,3.0,1150011100,NaN,NaN
4916,202201,1156013100100320001,11560131,A01,2.0,1156013100,NaN,NaN
7581,202202,1117010300100190001,11170103,A01,5.0,1117010300,NaN,NaN
10510,202202,1150011100106760000,11500111,A03,3.0,1150011100,NaN,NaN
10511,202202,1150011100106760000,11500111,A01,3.0,1150011100,NaN,NaN
11003,202202,1156013100100320001,11560131,A01,1.0,1156013100,NaN,NaN
13202,202203,1117010300100040000,11170103,A01,5.0,1117010300,NaN,NaN


In [27]:
temp[temp['LT_UNQ_NO_prefix']==1117010300] # 아예 없음...

,index,data_strd_ym,LT_UNQ_NO,INDUSTRY_CD,부동산거래대비유동인구,LT_UNQ_NO_prefix


In [28]:
notin_pop[notin_pop['data_strd_ym'].isna()]['LT_UNQ_NO_prefix'].unique()

array(['1117010300', '1150011100', '1156013100'], dtype=object)

In [29]:
temp[temp['LT_UNQ_NO_prefix'].str[:5].str.contains('11170')] # 같은 동인게 없어서, 시군구까지 같은 것들의 평균으로 대체

,index,data_strd_ym,LT_UNQ_NO,INDUSTRY_CD,부동산거래대비유동인구,LT_UNQ_NO_prefix
7864,15396,202203,1117010100100010043,A03,33.88,1117010100
7865,15397,202203,1117010100100020003,A13,21.12,1117010100
7866,15398,202203,1117010100100120004,A01,37.26,1117010100
7867,15399,202203,1117010100100190000,A02,23.53,1117010100
7868,15402,202203,1117010100100270011,A01,34.03,1117010100
...,...,...,...,...,...,...
2058641,22283,202303,1117013600102740017,A02,45.79,1117013600
2058642,22285,202303,1117013600102800001,A01,44.21,1117013600
2058643,22287,202303,1117013600103860001,A03,11.39,1117013600
2058644,22288,202303,1117013600103900002,A01,16.59,1117013600


In [30]:
notin_pop['LT_UNQ_NO_prefix2'] = notin_pop['LT_UNQ_NO'].astype(str).str[:5]
temp['LT_UNQ_NO_prefix2']= temp['LT_UNQ_NO'].astype(str).str[:5]

In [31]:
notin_pop

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,LT_UNQ_NO_prefix,data_strd_ym,부동산거래대비유동인구,LT_UNQ_NO_prefix2
0,202201,1111010200100570000,11110102,A05,4.0,1111010200,202203.0,35.098571,11110
1,202201,1111010400100530000,11110104,A05,5.0,1111010400,202203.0,27.218571,11110
2,202201,1111010600100350042,11110106,A01,2.0,1111010600,202203.0,41.603529,11110
3,202201,1111010600100370000,11110106,A02,4.0,1111010600,202203.0,41.603529,11110
4,202201,1111010600100490001,11110106,A02,2.0,1111010600,202203.0,41.603529,11110
...,...,...,...,...,...,...,...,...,...
1321,202302,1117013000101320006,11170130,A03,NaN,1117013000,202303.0,34.009218,11170
1322,202302,1156012900100610004,11560129,A04,NaN,1156012900,202303.0,37.916607,11560
1323,202302,1144012400102410098,11440124,A04,NaN,1144012400,202303.0,44.533655,11440
1324,202302,1130510300105680024,11305103,A02,NaN,1130510300,202303.0,49.861538,11305


In [32]:
temp

,index,data_strd_ym,LT_UNQ_NO,INDUSTRY_CD,부동산거래대비유동인구,LT_UNQ_NO_prefix,LT_UNQ_NO_prefix2
0,1,202203,1111010100100030100,A03,19.76,1111010100,11110
1,4,202203,1111010100100480000,A01,39.13,1111010100,11110
2,8,202203,1111010100100590001,A11,27.26,1111010100,11110
3,9,202203,1111010100100590005,A01,26.15,1111010100,11110
4,10,202203,1111010100100590005,A03,21.03,1111010100,11110
...,...,...,...,...,...,...,...
2562228,1168429,202303,5013032026104390007,A03,3.73,5013032026,50130
2562229,1168432,202303,5013032026104680001,A03,5.04,5013032026,50130
2562230,1168433,202303,5013032026105370003,A01,3.48,5013032026,50130
2562231,1168434,202303,5013032026108360001,A01,2.96,5013032026,50130


In [33]:
# 값 대체를 위한 groupby mean 계산해서 데이터프레임 생성
sub_df2 = temp.groupby(['LT_UNQ_NO_prefix2', 'data_strd_ym'])['부동산거래대비유동인구'].mean().reset_index()
sub_df2

,LT_UNQ_NO_prefix2,data_strd_ym,부동산거래대비유동인구
0,11110,202203,39.469810
1,11110,202206,42.496982
2,11110,202209,45.864392
3,11110,202212,47.285565
4,11110,202303,48.186468
...,...,...,...
1200,50130,202203,30.245530
1201,50130,202206,29.048950
1202,50130,202209,29.519856
1203,50130,202212,30.693670


In [34]:
sub_df2[sub_df2['LT_UNQ_NO_prefix2']=='11170']

,LT_UNQ_NO_prefix2,data_strd_ym,부동산거래대비유동인구
10,11170,202203,33.730577
11,11170,202206,32.841293
12,11170,202209,34.363889
13,11170,202212,34.760631
14,11170,202303,35.107087


In [35]:
# pop_df에서 필지명이 없어서 '부동산거래대비유동인구'가 결측값인 행들 추출
notin_pop2 = notin_pop[notin_pop['부동산거래대비유동인구'].isna()]
len(notin_pop2) # 75796

51

In [36]:
not_2201 = pd.merge(notin_pop2[notin_pop2['DATA_CRTR_YM'] <= 202203], sub_df2[sub_df2['data_strd_ym']==202203], on='LT_UNQ_NO_prefix2', how='left')
not_2202 = pd.merge(notin_pop2[(notin_pop2['DATA_CRTR_YM'] <= 202206) & (notin_pop2['DATA_CRTR_YM'] > 202203)], sub_df2[sub_df2['data_strd_ym']==202206], on='LT_UNQ_NO_prefix2', how='left')
not_2203 = pd.merge(notin_pop2[(notin_pop2['DATA_CRTR_YM'] <= 202209) & (notin_pop2['DATA_CRTR_YM'] > 202206)], sub_df2[sub_df2['data_strd_ym']==202209], on='LT_UNQ_NO_prefix2', how='left')
not_2204 = pd.merge(notin_pop2[(notin_pop2['DATA_CRTR_YM'] <= 202212) & (notin_pop2['DATA_CRTR_YM'] > 202209)], sub_df2[sub_df2['data_strd_ym']==202212], on='LT_UNQ_NO_prefix2', how='left')
not_2301 = pd.merge(notin_pop2[(notin_pop2['DATA_CRTR_YM'] <= 202303) & (notin_pop2['DATA_CRTR_YM'] > 202212)], sub_df2[sub_df2['data_strd_ym']==202303], on='LT_UNQ_NO_prefix2', how='left')

In [37]:
# 분기별 데이터 하나로 합치기
fill_pop = pd.concat([not_2201, not_2202, not_2203, not_2204, not_2301]).reset_index()
fill_pop.info() # 51개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              51 non-null     int64  
 1   DATA_CRTR_YM       51 non-null     int64  
 2   LT_UNQ_NO          51 non-null     int64  
 3   STDG_EMD_CD        51 non-null     int64  
 4   INDUSTRY_CD        51 non-null     object 
 5   SLS_GRD            51 non-null     float64
 6   LT_UNQ_NO_prefix   51 non-null     object 
 7   data_strd_ym_x     0 non-null      float64
 8   부동산거래대비유동인구_x      0 non-null      float64
 9   LT_UNQ_NO_prefix2  51 non-null     object 
 10  data_strd_ym_y     51 non-null     int64  
 11  부동산거래대비유동인구_y      51 non-null     float64
dtypes: float64(4), int64(5), object(3)
memory usage: 4.9+ KB


In [38]:
fill_pop.drop(columns=['index','부동산거래대비유동인구_x','data_strd_ym_x'], axis=1, inplace=True)
fill_pop.rename(columns={'부동산거래대비유동인구_y':'부동산거래대비유동인구','data_strd_ym_y':'data_strd_ym'}, inplace=True)

In [39]:
fill_pop.head()

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,LT_UNQ_NO_prefix,LT_UNQ_NO_prefix2,data_strd_ym,부동산거래대비유동인구
0,202201,1117010300100040000,11170103,A01,5.0,1117010300,11170,202203,33.730577
1,202201,1117010300100190001,11170103,A01,5.0,1117010300,11170,202203,33.730577
2,202201,1150011100106760000,11500111,A03,3.0,1150011100,11500,202203,50.599636
3,202201,1150011100106760000,11500111,A01,3.0,1150011100,11500,202203,50.599636
4,202201,1156013100100320001,11560131,A01,2.0,1156013100,11560,202203,49.150747


In [40]:
null_index = notin_pop['부동산거래대비유동인구'].isna()
notin_pop.loc[null_index, '부동산거래대비유동인구'] = fill_pop['부동산거래대비유동인구'].values

In [41]:
notin_pop.isnull().sum()

DATA_CRTR_YM            0
LT_UNQ_NO               0
STDG_EMD_CD             0
INDUSTRY_CD             0
SLS_GRD              1326
LT_UNQ_NO_prefix        0
data_strd_ym           51
부동산거래대비유동인구             0
LT_UNQ_NO_prefix2       0
dtype: int64

## 1-4) 최종적으로 합치기

In [42]:
null_index2 = pop_df['부동산거래대비유동인구'].isna()
pop_df.loc[null_index2, '부동산거래대비유동인구'] = notin_pop['부동산거래대비유동인구'].values

In [43]:
len(pop_df[pop_df['부동산거래대비유동인구'].isna()])

0

In [44]:
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176040 entries, 0 to 1176039
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   DATA_CRTR_YM      1176040 non-null  int64  
 1   LT_UNQ_NO         1176040 non-null  int64  
 2   STDG_EMD_CD       1176040 non-null  int64  
 3   INDUSTRY_CD       1176040 non-null  object 
 4   SLS_GRD           1156040 non-null  float64
 5   부동산거래대비유동인구       1176040 non-null  float64
 6   LT_UNQ_NO_prefix  1176040 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 62.8+ MB


In [45]:
pop_df

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100
1,202201,1111010100100480000,11110101,A01,2.0,39.13,1111010100
2,202201,1111010100100590001,11110101,A11,3.0,27.26,1111010100
3,202201,1111010100100590005,11110101,A03,2.0,21.03,1111010100
4,202201,1111010100100690000,11110101,A01,2.0,23.93,1111010100
...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,52.98,1130510100
1176036,202302,1135010300107450000,11350103,A13,NaN,36.41,1135010300
1176037,202302,1150010300103400017,11500103,A06,NaN,45.98,1150010300
1176038,202302,1144012700116070000,11440127,A03,NaN,56.97,1144012700


## 2. 공실률 대비 매매가 임대료 데이터 준비하고, pop_df에 이어서 붙이기

In [46]:
# 분기별 공실률 대비 매매가 임대료
cost2201 = pd.read_csv("공실률대비_매매가_임대료2201_csv.csv", sep='|') # 22년3월 집계
cost2202 = pd.read_csv("공실률대비_매매가_임대료2202_csv.csv", sep='|') # 22년 6월 집계
cost2203 = pd.read_csv("공실률대비_매매가_임대료2203_csv.csv", sep='|') # 22년 9월 집계
cost2204 = pd.read_csv("공실률대비_매매가_임대료2204_csv.csv", sep='|') # 22년 12월 집계
cost2301 = pd.read_csv("공실률대비_매매가_임대료2301_csv.csv", sep='|') # 23년 3월 집계

In [47]:
temp1 = cost2201[cost2201['induty_cd'].str.contains('A')]
temp2 = cost2202[cost2202['induty_cd'].str.contains('A')]
temp3 = cost2203[cost2203['induty_cd'].str.contains('A')]
temp4 = cost2204[cost2204['induty_cd'].str.contains('A')]
temp5 = cost2301[cost2301['induty_cd'].str.contains('A')]

In [48]:
temp = pd.concat([temp1, temp2,temp3, temp4, temp5])

In [49]:
len(temp1)+len(temp2)+len(temp3)+len(temp4)+ len(temp5) # len(temp)

1902767

In [50]:
temp.isnull().sum()

data_strd_ym      0
pnu               0
legaldong_cd      0
induty_cd         0
snp_price_scor    0
dtype: int64

## 2-1) 원본 데이터+test 데이터 합친 raw 데이터에 결합

In [51]:
temp.drop(['legaldong_cd'], axis=1, inplace=True)

In [52]:
temp.head()

,data_strd_ym,pnu,induty_cd,snp_price_scor
1,202203,1111010100100480000,A01,73.91
4,202203,1111010100100590001,A11,64.25
5,202203,1111010100100590005,A01,73.61
6,202203,1111010100100590005,A03,71.80
9,202203,1111010100100690000,A01,79.38


In [53]:
temp.rename(columns={'pnu':'LT_UNQ_NO' ,'induty_cd':'INDUSTRY_CD', 'snp_price_scor': '공실률대비매매가임대료'}, inplace=True)

In [54]:
temp.reset_index(inplace=True)
temp.drop(['index'], axis=1, inplace=True)

In [55]:
temp.drop_duplicates(subset=['data_strd_ym','LT_UNQ_NO','INDUSTRY_CD'], keep='first', inplace=True, ignore_index=False) 
temp #1902767에서 1888402됨

,data_strd_ym,LT_UNQ_NO,INDUSTRY_CD,공실률대비매매가임대료
0,202203,1111010100100480000,A01,73.91
1,202203,1111010100100590001,A11,64.25
2,202203,1111010100100590005,A01,73.61
3,202203,1111010100100590005,A03,71.80
4,202203,1111010100100690000,A01,79.38
...,...,...,...,...
1902762,202303,5013032025121620001,A01,98.23
1902763,202303,5013032026100370001,A01,99.21
1902764,202303,5013032026104230003,A01,99.19
1902765,202303,5013032026104300003,A03,98.98


In [56]:
pop_df

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100
1,202201,1111010100100480000,11110101,A01,2.0,39.13,1111010100
2,202201,1111010100100590001,11110101,A11,3.0,27.26,1111010100
3,202201,1111010100100590005,11110101,A03,2.0,21.03,1111010100
4,202201,1111010100100690000,11110101,A01,2.0,23.93,1111010100
...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,52.98,1130510100
1176036,202302,1135010300107450000,11350103,A13,NaN,36.41,1135010300
1176037,202302,1150010300103400017,11500103,A06,NaN,45.98,1150010300
1176038,202302,1144012700116070000,11440127,A03,NaN,56.97,1144012700


In [57]:
conditions = [
    (pop_df['DATA_CRTR_YM'] <= 202203), # 조건1
    ((pop_df['DATA_CRTR_YM'] >= 202204) & (pop_df['DATA_CRTR_YM'] <= 202206)), # 조건2
    ((pop_df['DATA_CRTR_YM'] >= 202207) & (pop_df['DATA_CRTR_YM'] <= 202209)), 
    ((pop_df['DATA_CRTR_YM'] >= 202210) & (pop_df['DATA_CRTR_YM'] <= 202212)), 
    ((pop_df['DATA_CRTR_YM'] >= 202301) & (pop_df['DATA_CRTR_YM'] <= 202303)), 
]

choices = [
    temp[temp['data_strd_ym'] == 202203],  # 조건 1에 대한 선택
    temp[temp['data_strd_ym'] == 202206],  # 조건 2에 대한 선택
    temp[temp['data_strd_ym'] == 202209],
    temp[temp['data_strd_ym'] == 202212],
    temp[temp['data_strd_ym'] == 202303],
]

price_df = pd.DataFrame()  # 빈 데이터프레임을 생성하여 결과를 저장합니다.

for i in range(len(conditions)):
    condition = conditions[i]
    selected_rows_A = pop_df[condition]
    selected_rows_B = choices[i]  # 조건에 따른 선택
    price_df_part = pd.merge(selected_rows_A, selected_rows_B, on=['LT_UNQ_NO', 'INDUSTRY_CD'], how='left')
    price_df = pd.concat([price_df, price_df_part], ignore_index=True)

In [58]:
price_df # 데이터 손실 없음

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,data_strd_ym,공실률대비매매가임대료
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,NaN,NaN
1,202201,1111010100100480000,11110101,A01,2.0,39.13,1111010100,202203.0,73.91
2,202201,1111010100100590001,11110101,A11,3.0,27.26,1111010100,202203.0,64.25
3,202201,1111010100100590005,11110101,A03,2.0,21.03,1111010100,202203.0,71.80
4,202201,1111010100100690000,11110101,A01,2.0,23.93,1111010100,202203.0,79.38
...,...,...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,52.98,1130510100,202303.0,66.38
1176036,202302,1135010300107450000,11350103,A13,NaN,36.41,1135010300,NaN,NaN
1176037,202302,1150010300103400017,11500103,A06,NaN,45.98,1150010300,202303.0,70.07
1176038,202302,1144012700116070000,11440127,A03,NaN,56.97,1144012700,202303.0,78.26


In [59]:
price_df.drop(['data_strd_ym'], axis=1, inplace=True)

## 2-2) 필지번호 매칭 안되는 것들 평균 넣어주기

In [60]:
# 'LT_UNQ_NO' 열에서 앞 10자리를 추출
temp['LT_UNQ_NO_prefix'] = temp['LT_UNQ_NO'].astype(str).str[:10]

In [61]:
# 값 대체를 위한 groupby mean 계산해서 데이터프레임 생성
sub_df = temp.groupby(['LT_UNQ_NO_prefix', 'data_strd_ym'])['공실률대비매매가임대료'].mean().reset_index()
sub_df

,LT_UNQ_NO_prefix,data_strd_ym,공실률대비매매가임대료
0,1111010100,202203,73.540769
1,1111010100,202206,70.803077
2,1111010100,202209,69.832308
3,1111010100,202212,67.564286
4,1111010100,202303,66.805833
...,...,...,...
44144,5013032026,202203,99.232500
44145,5013032026,202206,99.083333
44146,5013032026,202209,99.218000
44147,5013032026,202212,99.175000


In [62]:
# price_df에서 필지명이 없어서 '공실률대비매매가임대료'가 결측값인 행들 추출
not_merged = price_df[price_df['공실률대비매매가임대료'].isna()]
len(not_merged) # 75796

240444

In [63]:
not_2201 = pd.merge(not_merged[not_merged['DATA_CRTR_YM'] <= 202203], sub_df[sub_df['data_strd_ym']==202203], on='LT_UNQ_NO_prefix', how='left')
not_2202 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202206) & (not_merged['DATA_CRTR_YM'] > 202203)], sub_df[sub_df['data_strd_ym']==202206], on='LT_UNQ_NO_prefix', how='left')
not_2203 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202209) & (not_merged['DATA_CRTR_YM'] > 202206)], sub_df[sub_df['data_strd_ym']==202209], on='LT_UNQ_NO_prefix', how='left')
not_2204 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202212) & (not_merged['DATA_CRTR_YM'] > 202209)], sub_df[sub_df['data_strd_ym']==202212], on='LT_UNQ_NO_prefix', how='left')
not_2301 = pd.merge(not_merged[(not_merged['DATA_CRTR_YM'] <= 202303) & (not_merged['DATA_CRTR_YM'] > 202212)], sub_df[sub_df['data_strd_ym']==202303], on='LT_UNQ_NO_prefix', how='left')

In [64]:
# 분기별 데이터 하나로 합치기
notin_price = pd.concat([not_2201, not_2202, not_2203, not_2204, not_2301])
notin_price # 240444

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료_x,data_strd_ym,공실률대비매매가임대료_y
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,NaN,202203.0,73.540769
1,202201,1111010100100720000,11110101,A14,4.0,17.66,1111010100,NaN,202203.0,73.540769
2,202201,1111010100100750000,11110101,A01,2.0,27.27,1111010100,NaN,202203.0,73.540769
3,202201,1111010100101310000,11110101,A03,3.0,34.70,1111010100,NaN,202203.0,73.540769
4,202201,1111010200100010028,11110102,A03,4.0,23.09,1111010200,NaN,202203.0,71.255000
...,...,...,...,...,...,...,...,...,...,...
4239,202302,1171010700109150000,11710107,A03,NaN,40.07,1171010700,NaN,202303.0,69.175648
4240,202302,1147010100113160000,11470101,A04,NaN,18.81,1147010100,NaN,202303.0,63.217768
4241,202302,1117010700100430117,11170107,A01,NaN,35.54,1117010700,NaN,202303.0,76.873936
4242,202302,1171010900108950000,11710109,A06,NaN,48.76,1171010900,NaN,202303.0,68.660909


In [65]:
notin_price.drop(columns='공실률대비매매가임대료_x', axis=1, inplace=True)
notin_price.rename(columns={'공실률대비매매가임대료_y':'공실률대비매매가임대료'}, inplace=True)

## 1-3) 이 안에서 또 결측값 있어서 시군구 기준으로 평균낸 값 넣기

In [66]:
notin_price[notin_price['data_strd_ym'].isna()] # 588개.. 여기에는 test데이터도 있어서 대체하긴 해야할듯 

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,data_strd_ym,공실률대비매매가임대료
257,202201,1111012600100010000,11110126,A04,1.0,52.600000,1111012600,NaN,NaN
258,202201,1111012600100010000,11110126,A14,1.0,48.120000,1111012600,NaN,NaN
259,202201,1111012600100010000,11110126,A01,1.0,50.880000,1111012600,NaN,NaN
260,202201,1111012600100010000,11110126,A03,1.0,47.120000,1111012600,NaN,NaN
261,202201,1111012600100240000,11110126,A08,4.0,62.320000,1111012600,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3078,202302,1111012600100540000,11110126,A04,NaN,59.630000,1111012600,NaN,NaN
3528,202302,1111012600100010000,11110126,A04,NaN,59.630000,1111012600,NaN,NaN
3895,202302,1111012600100240000,11110126,A02,NaN,55.732727,1111012600,NaN,NaN
3904,202302,1111012600100240000,11110126,A06,NaN,46.730000,1111012600,NaN,NaN


In [67]:
notin_price[notin_price['data_strd_ym'].isna()]['LT_UNQ_NO_prefix'].unique()

array(['1111012600', '1111017900', '1114017200', '1117010300',
       '1117010600', '1141011400', '1141011500', '1150011100',
       '1150011300', '1153011200', '1156013100'], dtype=object)

In [68]:
notin_price['LT_UNQ_NO_prefix2'] = notin_price['LT_UNQ_NO'].astype(str).str[:5]
temp['LT_UNQ_NO_prefix2'] = temp['LT_UNQ_NO'].astype(str).str[:5]

In [69]:
# 값 대체를 위한 groupby mean 계산해서 데이터프레임 생성
sub_df2 = temp.groupby(['LT_UNQ_NO_prefix2', 'data_strd_ym'])['공실률대비매매가임대료'].mean().reset_index()
sub_df2

,LT_UNQ_NO_prefix2,data_strd_ym,공실률대비매매가임대료
0,11110,202203,75.015572
1,11110,202206,73.862203
2,11110,202209,73.213352
3,11110,202212,71.828992
4,11110,202303,71.670828
...,...,...,...
1200,50130,202203,95.752109
1201,50130,202206,96.168666
1202,50130,202209,95.738147
1203,50130,202212,95.801739


In [70]:
# price_df에서 필지명이 없어서 '공실률대비매매가임대료'가 결측값인 행들 추출
notin_price2 = notin_price[notin_price['공실률대비매매가임대료'].isna()]
len(notin_price2) # 75796

588

In [71]:
not_2201 = pd.merge(notin_price2[notin_price2['DATA_CRTR_YM'] <= 202203], sub_df2[sub_df2['data_strd_ym']==202203], on='LT_UNQ_NO_prefix2', how='left')
not_2202 = pd.merge(notin_price2[(notin_price2['DATA_CRTR_YM'] <= 202206) & (notin_price2['DATA_CRTR_YM'] > 202203)], sub_df2[sub_df2['data_strd_ym']==202206], on='LT_UNQ_NO_prefix2', how='left')
not_2203 = pd.merge(notin_price2[(notin_price2['DATA_CRTR_YM'] <= 202209) & (notin_price2['DATA_CRTR_YM'] > 202206)], sub_df2[sub_df2['data_strd_ym']==202209], on='LT_UNQ_NO_prefix2', how='left')
not_2204 = pd.merge(notin_price2[(notin_price2['DATA_CRTR_YM'] <= 202212) & (notin_price2['DATA_CRTR_YM'] > 202209)], sub_df2[sub_df2['data_strd_ym']==202212], on='LT_UNQ_NO_prefix2', how='left')
not_2301 = pd.merge(notin_price2[(notin_price2['DATA_CRTR_YM'] <= 202303) & (notin_price2['DATA_CRTR_YM'] > 202212)], sub_df2[sub_df2['data_strd_ym']==202303], on='LT_UNQ_NO_prefix2', how='left')

In [72]:
# 분기별 데이터 하나로 합치기
fill_price = pd.concat([not_2201, not_2202, not_2203, not_2204, not_2301]).reset_index()
fill_price.info() # 588개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              588 non-null    int64  
 1   DATA_CRTR_YM       588 non-null    int64  
 2   LT_UNQ_NO          588 non-null    int64  
 3   STDG_EMD_CD        588 non-null    int64  
 4   INDUSTRY_CD        588 non-null    object 
 5   SLS_GRD            572 non-null    float64
 6   부동산거래대비유동인구        588 non-null    float64
 7   LT_UNQ_NO_prefix   588 non-null    object 
 8   data_strd_ym_x     0 non-null      float64
 9   공실률대비매매가임대료_x      0 non-null      float64
 10  LT_UNQ_NO_prefix2  588 non-null    object 
 11  data_strd_ym_y     588 non-null    int64  
 12  공실률대비매매가임대료_y      588 non-null    float64
dtypes: float64(5), int64(5), object(3)
memory usage: 59.8+ KB


In [73]:
fill_price.drop(columns=['index','공실률대비매매가임대료_x','data_strd_ym_x'], axis=1, inplace=True)
fill_price.rename(columns={'공실률대비매매가임대료_y':'공실률대비매매가임대료','data_strd_ym_y':'data_strd_ym'}, inplace=True)

In [74]:
fill_price.head()

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,LT_UNQ_NO_prefix2,data_strd_ym,공실률대비매매가임대료
0,202201,1111012600100010000,11110126,A04,1.0,52.60,1111012600,11110,202203,75.015572
1,202201,1111012600100010000,11110126,A14,1.0,48.12,1111012600,11110,202203,75.015572
2,202201,1111012600100010000,11110126,A01,1.0,50.88,1111012600,11110,202203,75.015572
3,202201,1111012600100010000,11110126,A03,1.0,47.12,1111012600,11110,202203,75.015572
4,202201,1111012600100240000,11110126,A08,4.0,62.32,1111012600,11110,202203,75.015572


In [75]:
null_index = notin_price['공실률대비매매가임대료'].isna()
notin_price.loc[null_index, '공실률대비매매가임대료'] = fill_price['공실률대비매매가임대료'].values

## 2-4) 최종적으로 합치기

In [76]:
null_index2 = price_df['공실률대비매매가임대료'].isna()
price_df.loc[null_index2, '공실률대비매매가임대료'] = notin_price['공실률대비매매가임대료'].values

In [77]:
len(price_df[price_df['공실률대비매매가임대료'].isna()])

0

In [78]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176040 entries, 0 to 1176039
Data columns (total 8 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   DATA_CRTR_YM      1176040 non-null  int64  
 1   LT_UNQ_NO         1176040 non-null  int64  
 2   STDG_EMD_CD       1176040 non-null  int64  
 3   INDUSTRY_CD       1176040 non-null  object 
 4   SLS_GRD           1156040 non-null  float64
 5   부동산거래대비유동인구       1176040 non-null  float64
 6   LT_UNQ_NO_prefix  1176040 non-null  object 
 7   공실률대비매매가임대료       1176040 non-null  float64
dtypes: float64(3), int64(3), object(2)
memory usage: 71.8+ MB


In [79]:
price_df

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,73.540769
1,202201,1111010100100480000,11110101,A01,2.0,39.13,1111010100,73.910000
2,202201,1111010100100590001,11110101,A11,3.0,27.26,1111010100,64.250000
3,202201,1111010100100590005,11110101,A03,2.0,21.03,1111010100,71.800000
4,202201,1111010100100690000,11110101,A01,2.0,23.93,1111010100,79.380000
...,...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,52.98,1130510100,66.380000
1176036,202302,1135010300107450000,11350103,A13,NaN,36.41,1135010300,63.328367
1176037,202302,1150010300103400017,11500103,A06,NaN,45.98,1150010300,70.070000
1176038,202302,1144012700116070000,11440127,A03,NaN,56.97,1144012700,78.260000


## 3. 젠트리피케이션 위험지수 이어서 합치기

In [113]:
# 매달 젠트리피케이션 위험지수
risk2201 = pd.read_csv("젠트리피케이션_위험지수2201_csv.csv", sep='|') 
risk2202 = pd.read_csv("젠트리피케이션_위험지수2202_csv.csv", sep='|') 
risk2203 = pd.read_csv("젠트리피케이션_위험지수2203_csv.csv", sep='|') 
risk2204 = pd.read_csv("젠트리피케이션_위험지수2204_csv.csv", sep='|') 
risk2205 = pd.read_csv("젠트리피케이션_위험지수2205_csv.csv", sep='|') 
risk2206 = pd.read_csv("젠트리피케이션_위험지수2206_csv.csv", sep='|') 
risk2207 = pd.read_csv("젠트리피케이션_위험지수2207_csv.csv", sep='|') 
risk2208 = pd.read_csv("젠트리피케이션_위험지수2208_csv.csv", sep='|') 
risk2209 = pd.read_csv("젠트리피케이션_위험지수2209_csv.csv", sep='|') 
risk2210 = pd.read_csv("젠트리피케이션_위험지수2210_csv.csv", sep='|') 
risk2211 = pd.read_csv("젠트리피케이션_위험지수2211_csv.csv", sep='|') 
risk2212 = pd.read_csv("젠트리피케이션_위험지수2212_csv.csv", sep='|') 
risk2301 = pd.read_csv("젠트리피케이션_위험지수2301_csv.csv", sep='|') 
risk2302 = pd.read_csv("젠트리피케이션_위험지수2302_csv.csv", sep='|') 

In [120]:
temp = pd.concat([risk2201,risk2202,risk2203,risk2204,risk2205,risk2206,risk2207,risk2208,risk2209,risk2210,risk2211,risk2212,risk2301,risk2302])

In [121]:
temp.isnull().sum()

data_strd_ym    0
pnu             0
legaldong_cd    0
induty_cd       0
gtfc_scor       0
dtype: int64

In [122]:
temp

,data_strd_ym,pnu,legaldong_cd,induty_cd,gtfc_scor
0,202201,1111010100100060003,11110101,B22,44.97
1,202201,1111010100100080004,11110101,C01,48.98
2,202201,1111010100100480000,11110101,A01,59.42
3,202201,1111010100100500031,11110101,C05,49.49
4,202201,1111010100100570056,11110101,C05,49.30
...,...,...,...,...,...
879946,202302,5013032026104150001,50130320,C07,43.40
879947,202302,5013032026104230003,50130320,A01,40.92
879948,202302,5013032026104390007,50130320,A03,43.72
879949,202302,5013032026104480011,50130320,A05,49.25


## 3-1) 원본 데이터+test 데이터 합친 raw 데이터에 결합

In [123]:
temp.drop(['legaldong_cd'], axis=1, inplace=True)

In [124]:
temp.head()

,data_strd_ym,pnu,induty_cd,gtfc_scor
0,202201,1111010100100060003,B22,44.97
1,202201,1111010100100080004,C01,48.98
2,202201,1111010100100480000,A01,59.42
3,202201,1111010100100500031,C05,49.49
4,202201,1111010100100570056,C05,49.30


In [126]:
temp.rename(columns={'data_strd_ym':'DATA_CRTR_YM','pnu':'LT_UNQ_NO' ,'induty_cd':'INDUSTRY_CD', 'gtfc_scor': '젠트리피케이션'}, inplace=True)

In [127]:
temp.reset_index(inplace=True)
temp.drop(['index'], axis=1, inplace=True)

In [128]:
temp # 12285597개 

,DATA_CRTR_YM,LT_UNQ_NO,INDUSTRY_CD,젠트리피케이션
0,202201,1111010100100060003,B22,44.97
1,202201,1111010100100080004,C01,48.98
2,202201,1111010100100480000,A01,59.42
3,202201,1111010100100500031,C05,49.49
4,202201,1111010100100570056,C05,49.30
...,...,...,...,...
12285592,202302,5013032026104150001,C07,43.40
12285593,202302,5013032026104230003,A01,40.92
12285594,202302,5013032026104390007,A03,43.72
12285595,202302,5013032026104480011,A05,49.25


In [130]:
temp.drop_duplicates(subset=['DATA_CRTR_YM','LT_UNQ_NO','INDUSTRY_CD'], keep='first', inplace=True, ignore_index=False) 
len(temp) # 똑같이 12285597

12285597

In [131]:
risk_df = pd.merge(price_df, temp, on=['DATA_CRTR_YM','LT_UNQ_NO', 'INDUSTRY_CD'], how='left')

In [132]:
risk_df # 데이터 손실 없음

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,젠트리피케이션
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,73.540769,NaN
1,202201,1111010100100480000,11110101,A01,2.0,39.13,1111010100,73.910000,59.42
2,202201,1111010100100590001,11110101,A11,3.0,27.26,1111010100,64.250000,48.86
3,202201,1111010100100590005,11110101,A03,2.0,21.03,1111010100,71.800000,46.78
4,202201,1111010100100690000,11110101,A01,2.0,23.93,1111010100,79.380000,50.58
...,...,...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,52.98,1130510100,66.380000,49.16
1176036,202302,1135010300107450000,11350103,A13,NaN,36.41,1135010300,63.328367,NaN
1176037,202302,1150010300103400017,11500103,A06,NaN,45.98,1150010300,70.070000,59.29
1176038,202302,1144012700116070000,11440127,A03,NaN,56.97,1144012700,78.260000,46.62


## 3-2) 필지번호 매칭 안되는 것들 평균 넣어주기

In [133]:
# 'LT_UNQ_NO' 열에서 앞 10자리를 추출
temp['LT_UNQ_NO_prefix'] = temp['LT_UNQ_NO'].astype(str).str[:10]

In [134]:
# 값 대체를 위한 groupby mean 계산해서 데이터프레임 생성
sub_df = temp.groupby(['LT_UNQ_NO_prefix', 'DATA_CRTR_YM'])['젠트리피케이션'].mean().reset_index()
sub_df

,LT_UNQ_NO_prefix,DATA_CRTR_YM,젠트리피케이션
0,1111010100,202201,51.190526
1,1111010100,202202,48.537778
2,1111010100,202203,48.590000
3,1111010100,202204,48.444737
4,1111010100,202205,48.696667
...,...,...,...
158961,5013032026,202210,48.060000
158962,5013032026,202211,48.853333
158963,5013032026,202212,48.747500
158964,5013032026,202301,46.365000


In [135]:
# risk_df에서 필지명이 없어서 '젠트리피케이션'가 결측값인 행들 추출
not_merged = risk_df[risk_df['젠트리피케이션'].isna()]
len(not_merged) # 255295

255295

In [136]:
notin_risk = pd.merge(not_merged, sub_df, on=['DATA_CRTR_YM','LT_UNQ_NO_prefix'], how='left')
notin_risk

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,젠트리피케이션_x,젠트리피케이션_y
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,73.540769,NaN,51.190526
1,202201,1111010100100720000,11110101,A14,4.0,17.66,1111010100,73.540769,NaN,51.190526
2,202201,1111010100100750000,11110101,A01,2.0,27.27,1111010100,73.540769,NaN,51.190526
3,202201,1111010100100970000,11110101,A03,2.0,27.05,1111010100,71.640000,NaN,51.190526
4,202201,1111010100101310000,11110101,A03,3.0,34.70,1111010100,73.540769,NaN,51.190526
...,...,...,...,...,...,...,...,...,...,...
255290,202302,1168010800102760001,11680108,A01,NaN,35.96,1168010800,76.636737,NaN,50.525767
255291,202302,1171010700109150000,11710107,A03,NaN,40.07,1171010700,69.175648,NaN,49.989526
255292,202302,1147010100113160000,11470101,A04,NaN,18.81,1147010100,63.217768,NaN,49.750585
255293,202302,1171010900108950000,11710109,A06,NaN,48.76,1171010900,68.660909,NaN,49.170000


In [137]:
notin_risk.drop(columns='젠트리피케이션_x', axis=1, inplace=True)
notin_risk.rename(columns={'젠트리피케이션_y':'젠트리피케이션'}, inplace=True)

## 3-3) 이 안에서 또 결측값 있어서 시군구 기준으로 평균낸 값 넣기

In [138]:
notin_risk[notin_risk['젠트리피케이션'].isna()] # 295개.. 여기에는 test데이터도 있어서 대체하긴 해야할듯 

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,젠트리피케이션
230,202201,1111012600100010000,11110126,A04,1.0,52.600000,1111012600,75.015572,NaN
231,202201,1111012600100010000,11110126,A14,1.0,48.120000,1111012600,75.015572,NaN
232,202201,1111012600100010000,11110126,A01,1.0,50.880000,1111012600,75.015572,NaN
233,202201,1111012600100010000,11110126,A03,1.0,47.120000,1111012600,75.015572,NaN
234,202201,1111012600100240000,11110126,A08,4.0,62.320000,1111012600,75.015572,NaN
...,...,...,...,...,...,...,...,...,...
254681,202302,1111012600100540000,11110126,A04,NaN,59.630000,1111012600,71.670828,NaN
254900,202302,1111012600100010000,11110126,A04,NaN,59.630000,1111012600,71.670828,NaN
255115,202302,1111012600100240000,11110126,A02,NaN,55.732727,1111012600,71.670828,NaN
255120,202302,1111012600100240000,11110126,A06,NaN,46.730000,1111012600,71.670828,NaN


In [139]:
notin_risk[notin_risk['젠트리피케이션'].isna()]['LT_UNQ_NO_prefix'].unique()

array(['1111012600', '1117010300', '1117010600', '1141011500',
       '1150011100', '1150011300', '1156013100'], dtype=object)

In [140]:
notin_risk['LT_UNQ_NO_prefix2'] = notin_risk['LT_UNQ_NO'].astype(str).str[:5]
temp['LT_UNQ_NO_prefix2'] = temp['LT_UNQ_NO'].astype(str).str[:5]

In [141]:
temp

,DATA_CRTR_YM,LT_UNQ_NO,INDUSTRY_CD,젠트리피케이션,LT_UNQ_NO_prefix,LT_UNQ_NO_prefix2
0,202201,1111010100100060003,B22,44.97,1111010100,11110
1,202201,1111010100100080004,C01,48.98,1111010100,11110
2,202201,1111010100100480000,A01,59.42,1111010100,11110
3,202201,1111010100100500031,C05,49.49,1111010100,11110
4,202201,1111010100100570056,C05,49.30,1111010100,11110
...,...,...,...,...,...,...
12285592,202302,5013032026104150001,C07,43.40,5013032026,50130
12285593,202302,5013032026104230003,A01,40.92,5013032026,50130
12285594,202302,5013032026104390007,A03,43.72,5013032026,50130
12285595,202302,5013032026104480011,A05,49.25,5013032026,50130


In [142]:
# 값 대체를 위한 groupby mean 계산해서 데이터프레임 생성
sub_df2 = temp.groupby(['LT_UNQ_NO_prefix2', 'DATA_CRTR_YM'])['젠트리피케이션'].mean().reset_index()
sub_df2

,LT_UNQ_NO_prefix2,DATA_CRTR_YM,젠트리피케이션
0,11110,202201,50.371309
1,11110,202202,49.902243
2,11110,202203,49.553360
3,11110,202204,49.779368
4,11110,202205,49.699615
...,...,...,...
3495,50130,202210,49.406907
3496,50130,202211,49.431767
3497,50130,202212,49.459975
3498,50130,202301,48.732068


In [143]:
# risk_df에서 필지명이 없어서 '젠트리피케이션'가 결측값인 행들 추출
notin_risk2 = notin_risk[notin_risk['젠트리피케이션'].isna()]
len(notin_risk2) # 295

295

In [144]:
fill_risk= pd.merge(notin_risk2,sub_df2, on=['DATA_CRTR_YM','LT_UNQ_NO_prefix2'], how='left')
fill_risk.info() # 295개 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295 entries, 0 to 294
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DATA_CRTR_YM       295 non-null    int64  
 1   LT_UNQ_NO          295 non-null    int64  
 2   STDG_EMD_CD        295 non-null    int64  
 3   INDUSTRY_CD        295 non-null    object 
 4   SLS_GRD            283 non-null    float64
 5   부동산거래대비유동인구        295 non-null    float64
 6   LT_UNQ_NO_prefix   295 non-null    object 
 7   공실률대비매매가임대료        295 non-null    float64
 8   젠트리피케이션_x          0 non-null      float64
 9   LT_UNQ_NO_prefix2  295 non-null    object 
 10  젠트리피케이션_y          295 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 27.7+ KB


In [145]:
fill_risk.drop(columns=['젠트리피케이션_x'], axis=1, inplace=True)
fill_risk.rename(columns={'젠트리피케이션_y':'젠트리피케이션'}, inplace=True)

In [146]:
fill_risk.head()

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,LT_UNQ_NO_prefix2,젠트리피케이션
0,202201,1111012600100010000,11110126,A04,1.0,52.60,1111012600,75.015572,11110,50.371309
1,202201,1111012600100010000,11110126,A14,1.0,48.12,1111012600,75.015572,11110,50.371309
2,202201,1111012600100010000,11110126,A01,1.0,50.88,1111012600,75.015572,11110,50.371309
3,202201,1111012600100010000,11110126,A03,1.0,47.12,1111012600,75.015572,11110,50.371309
4,202201,1111012600100240000,11110126,A08,4.0,62.32,1111012600,75.015572,11110,50.371309


In [147]:
null_index2 = notin_risk['젠트리피케이션'].isna()
notin_risk.loc[null_index2, '젠트리피케이션'] = fill_risk['젠트리피케이션'].values

In [148]:
notin_risk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255295 entries, 0 to 255294
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DATA_CRTR_YM       255295 non-null  int64  
 1   LT_UNQ_NO          255295 non-null  int64  
 2   STDG_EMD_CD        255295 non-null  int64  
 3   INDUSTRY_CD        255295 non-null  object 
 4   SLS_GRD            253044 non-null  float64
 5   부동산거래대비유동인구        255295 non-null  float64
 6   LT_UNQ_NO_prefix   255295 non-null  object 
 7   공실률대비매매가임대료        255295 non-null  float64
 8   젠트리피케이션            255295 non-null  float64
 9   LT_UNQ_NO_prefix2  255295 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 21.4+ MB


## 3-4) 최종적으로 합치기

In [149]:
risk_df[risk_df['젠트리피케이션'].isna()]

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,젠트리피케이션
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,73.540769,NaN
5,202201,1111010100100720000,11110101,A14,4.0,17.66,1111010100,73.540769,NaN
6,202201,1111010100100750000,11110101,A01,2.0,27.27,1111010100,73.540769,NaN
9,202201,1111010100100970000,11110101,A03,2.0,27.05,1111010100,71.640000,NaN
13,202201,1111010100101310000,11110101,A03,3.0,34.70,1111010100,73.540769,NaN
...,...,...,...,...,...,...,...,...,...
1175989,202302,1168010800102760001,11680108,A01,NaN,35.96,1168010800,76.636737,NaN
1175996,202302,1171010700109150000,11710107,A03,NaN,40.07,1171010700,69.175648,NaN
1176008,202302,1147010100113160000,11470101,A04,NaN,18.81,1147010100,63.217768,NaN
1176027,202302,1171010900108950000,11710109,A06,NaN,48.76,1171010900,68.660909,NaN


In [150]:
len(notin_risk['젠트리피케이션'])

255295

In [151]:
null_index2 = risk_df['젠트리피케이션'].isna()
risk_df.loc[null_index2, '젠트리피케이션'] = notin_risk['젠트리피케이션'].values

In [152]:
len(risk_df[risk_df['젠트리피케이션'].isna()])

0

In [153]:
risk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176040 entries, 0 to 1176039
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   DATA_CRTR_YM      1176040 non-null  int64  
 1   LT_UNQ_NO         1176040 non-null  int64  
 2   STDG_EMD_CD       1176040 non-null  int64  
 3   INDUSTRY_CD       1176040 non-null  object 
 4   SLS_GRD           1156040 non-null  float64
 5   부동산거래대비유동인구       1176040 non-null  float64
 6   LT_UNQ_NO_prefix  1176040 non-null  object 
 7   공실률대비매매가임대료       1176040 non-null  float64
 8   젠트리피케이션           1176040 non-null  float64
dtypes: float64(4), int64(3), object(2)
memory usage: 89.7+ MB


In [154]:
risk_df

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,젠트리피케이션
0,202201,1111010100100030100,11110101,A03,4.0,19.76,1111010100,73.540769,51.190526
1,202201,1111010100100480000,11110101,A01,2.0,39.13,1111010100,73.910000,59.420000
2,202201,1111010100100590001,11110101,A11,3.0,27.26,1111010100,64.250000,48.860000
3,202201,1111010100100590005,11110101,A03,2.0,21.03,1111010100,71.800000,46.780000
4,202201,1111010100100690000,11110101,A01,2.0,23.93,1111010100,79.380000,50.580000
...,...,...,...,...,...,...,...,...,...
1176035,202302,1130510100100830004,11305101,A03,NaN,52.98,1130510100,66.380000,49.160000
1176036,202302,1135010300107450000,11350103,A13,NaN,36.41,1135010300,63.328367,48.943359
1176037,202302,1150010300103400017,11500103,A06,NaN,45.98,1150010300,70.070000,59.290000
1176038,202302,1144012700116070000,11440127,A03,NaN,56.97,1144012700,78.260000,46.620000


# 4. CPI까지 합치기

In [155]:
CPI = pd.read_csv("생활물가지수_20230825094528.csv")
raw_data = pd.read_csv('data_given.csv', encoding='cp949')
test = pd.read_excel('submission.xlsx')

In [156]:
test_2301 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]
raw_data = pd.concat([raw_data,test_2301, test_2302])

<ipython-input-156-cd89228ea372>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
<ipython-input-156-cd89228ea372>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [157]:
CPI.rename(columns={'시점':'DATA_CRTR_YM'}, inplace=True)
CPI.drop(['식품','식품 이외','전월세','전·월세포함 생활물가지수'], axis=1, inplace=True)
CPI['DATA_CRTR_YM'] = CPI['DATA_CRTR_YM'].str.replace(' ', '').str.replace('.', '')
CPI= CPI.astype({'DATA_CRTR_YM':'int'})
CPI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DATA_CRTR_YM  15 non-null     int64  
 1   생활물가지수        15 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 368.0 bytes


<ipython-input-157-31531a417cc9>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  CPI['DATA_CRTR_YM'] = CPI['DATA_CRTR_YM'].str.replace(' ', '').str.replace('.', '')


In [158]:
CPI_df = pd.merge(raw_data, CPI, on='DATA_CRTR_YM', how='left')

In [159]:
raw_data = pd.merge(CPI_df, risk_df, on=['DATA_CRTR_YM','LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'SLS_GRD'])
# 데이터 손실 없음

In [160]:
raw_data.head()

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,생활물가지수,부동산거래대비유동인구,LT_UNQ_NO_prefix,공실률대비매매가임대료,젠트리피케이션
0,202201,1111010100100030100,11110101,A03,4.0,104.55,19.76,1111010100,73.540769,51.190526
1,202201,1111010100100480000,11110101,A01,2.0,104.55,39.13,1111010100,73.910000,59.420000
2,202201,1111010100100590001,11110101,A11,3.0,104.55,27.26,1111010100,64.250000,48.860000
3,202201,1111010100100590005,11110101,A03,2.0,104.55,21.03,1111010100,71.800000,46.780000
4,202201,1111010100100690000,11110101,A01,2.0,104.55,23.93,1111010100,79.380000,50.580000


In [161]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176040 entries, 0 to 1176039
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   DATA_CRTR_YM      1176040 non-null  int64  
 1   LT_UNQ_NO         1176040 non-null  int64  
 2   STDG_EMD_CD       1176040 non-null  int64  
 3   INDUSTRY_CD       1176040 non-null  object 
 4   SLS_GRD           1156040 non-null  float64
 5   생활물가지수            1176040 non-null  float64
 6   부동산거래대비유동인구       1176040 non-null  float64
 7   LT_UNQ_NO_prefix  1176040 non-null  object 
 8   공실률대비매매가임대료       1176040 non-null  float64
 9   젠트리피케이션           1176040 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 98.7+ MB


In [162]:
raw_data.to_csv("raw_data_윤정.csv", encoding="cp949", index=False)